In [1]:
from bs4 import BeautifulSoup as B
from selenium import webdriver
import pandas as pd
import time
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
websitelist=[]
userlist=[]

In [2]:
#for Mozillza Firefox
#download https://github.com/mozilla/geckodriver/releases
# store this file in jyputer notebook location

In [4]:
driver=webdriver.Chrome()

In [5]:
url='https://www.webmd.com/drugs/2/index'
driver.get(url)
soup=B(driver.page_source,'html.parser')


In [6]:
link=soup.find_all('a',{'class':"common-result-review"})

In [7]:
link

[<a class="common-result-review" data-metrics-link="cmndrg-rev" href="/drugs/drugreview-64439-abilify.aspx?drugid=64439&amp;drugname=abilify">Read Reviews (1815)</a>,
 <a class="common-result-review" data-metrics-link="cmndrg-rev" href="/drugs/drugreview-17410-actos.aspx?drugid=17410&amp;drugname=actos">Read Reviews (660)</a>,
 <a class="common-result-review" data-metrics-link="cmndrg-rev" href="/drugs/drugreview-941-acyclovir.aspx?drugid=941&amp;drugname=acyclovir">Read Reviews (371)</a>,
 <a class="common-result-review" data-metrics-link="cmndrg-rev" href="/drugs/drugreview-63164-adderall-xr.aspx?drugid=63164&amp;drugname=adderall-xr">Read Reviews (647)</a>,
 <a class="common-result-review" data-metrics-link="cmndrg-rev" href="/drugs/drugreview-8610-allopurinol.aspx?drugid=8610&amp;drugname=allopurinol">Read Reviews (302)</a>,
 <a class="common-result-review" data-metrics-link="cmndrg-rev" href="/drugs/drugreview-9690-ambien.aspx?drugid=9690&amp;drugname=ambien">Read Reviews (1478)</

In [8]:
for l in link:
    a=l.get('href')
    websitelist.append(a)

web=pd.DataFrame(websitelist,columns=['website'])

In [9]:
web['website']='https://www.webmd.com/'+web['website']

In [10]:
web.website[:]

0     https://www.webmd.com//drugs/drugreview-64439-...
1     https://www.webmd.com//drugs/drugreview-17410-...
2     https://www.webmd.com//drugs/drugreview-941-ac...
3     https://www.webmd.com//drugs/drugreview-63164-...
4     https://www.webmd.com//drugs/drugreview-8610-a...
                            ...                        
91    https://www.webmd.com//drugs/drugreview-163054...
92    https://www.webmd.com//drugs/drugreview-148324...
93    https://www.webmd.com//drugs/drugreview-76851-...
94    https://www.webmd.com//drugs/drugreview-9824-x...
95    https://www.webmd.com//drugs/drugreview-30-zof...
Name: website, Length: 96, dtype: object

In [11]:
def get_review(soup):
    reviews=soup.find_all('div',{'class':'userPost'})
    m=1
    for item in reviews:
        
        review={
        'Medicine':soup.title.text.replace('Reviews and User Ratings: Effectiveness, Ease of Use, and Satisfaction','').strip(),
        'Date':item.find('div',{'class':'date'}).text.strip(),
        'Condition':item.find('div',{'class':'conditionInfo'}).text.strip(),
        'Effectiveness':[float(i) for i in item.find('div',{'class':"catRatings firstEl clearfix"}).get_text().strip() if i.isdigit()],
        'EaseofUse':[float(i) for i in item.find('div',{'class':"catRatings clearfix"}).get_text().strip() if i.isdigit()],
        'Satisfaction':[float(i) for i in item.find('div',{'class':"catRatings lastEl clearfix"}).get_text().strip() if i.isdigit()],
        'Review':item.find('p',{'id':'comFull{}'.format(m)}).text.replace('Hide Full Comment','').replace('Comment:','').replace('\n','')
        }
        userlist.append(review)
        m=m+1

In [12]:
def page_review(site,j):
    url=site
    i=1
    k=1
    while i==1:
        driver.get(url) 
        if k==1:
            driver.find_element_by_xpath("//select[@id='conditionFilter']/option[@value='-1']").click()
        soupp=B(driver.page_source,'html.parser')
        get_review(soupp)
        link=soupp.find_all('a',{'onclick':"ctrs('dr-pagenum_next')"})
        print('Medicine',j,' :page',k)
        #time.sleep(5)

        if not link:
            i=0
            
        else:
            a=link[0].get('href')
            url='https://www.webmd.com/'+a
            k=k+1
           
    

In [13]:
def website_main(data):
    j=1
    for site in data.website[1:]:
        page_review(site,j)
        j=j+1
    
    

In [16]:
userlist=[]
website_main(web)

Medicine 1  :page 1
Medicine 1  :page 2
Medicine 1  :page 3
Medicine 1  :page 4
Medicine 1  :page 5


KeyboardInterrupt: 

In [17]:
A=pd.DataFrame(userlist)

In [18]:
A

,Medicine,Date,Condition,Effectiveness,EaseofUse,Satisfaction,Review
0,Actos oral,7/31/2020 2:32:15 PM,Condition: Type 2 Diabetes Mellitus,[5.0],[5.0],[5.0],No Script or health Insurance needed to place ...
1,Actos oral,6/2/2020 7:53:34 PM,Condition: Other,[5.0],[5.0],[5.0],When i first heard about herbal remedy i never...
2,Actos oral,12/7/2019 3:36:39 AM,Condition: Type 2 Diabetes Mellitus,[1.0],[1.0],[1.0],Absolutely no control over sugars since taking...
3,Actos oral,11/11/2019 9:17:24 AM,Condition: Other,[3.0],[4.0],[3.0],good drug
4,Actos oral,10/29/2019 11:38:07 AM,Condition: Type 2 Diabetes Mellitus,[3.0],[3.0],[3.0],
5,Actos oral,10/29/2019 8:31:23 AM,Condition: Other,[3.0],[3.0],[3.0],
6,Actos oral,7/20/2019 4:05:04 AM,Condition: Other,[5.0],[5.0],[5.0],"I offer top quality medications, we have mul..."
7,Actos oral,5/8/2019 3:08:12 PM,Condition: Type 2 Diabetes Mellitus,[2.0],[2.0],[1.0],"I am exhausted. I am usually very active, but ..."
8,Actos oral,4/12/2019 4:35:25 PM,Condition: Type 2 Diabetes Mellitus,[3.0],[3.0],[1.0],Been on for 3 months...numbers came down but w...
9,Actos oral,2/10/2019 9:01:19 AM,Condition: Type 2 Diabetes Mellitus,[5.0],[5.0],[3.0],Too many problems reported with this medication


In [19]:
A.to_csv('Drug.csv',index=False)